In [1]:
import pymysql
import pandas as pd

# Configuración BD (misma que tienes en tu código original)
DB_HOST = "10.254.33.138"
DB_USER = "compensaciones_rrhh"
DB_PASSWORD = "_Cramercomp2025_"
DB_NAME = "rrhh_app"

In [2]:
def conectar_y_extraer_datos():
    """
    Conecta a la base de datos y extrae las columnas específicas en un DataFrame
    """
    try:
        print("🚀 Conectando a MySQL...")
        
        # Establecer conexión
        conexion = pymysql.connect(
            host=DB_HOST,
            user=DB_USER,
            password=DB_PASSWORD,
            database=DB_NAME,
            charset='utf8mb4'
        )
        
        print(f"✅ Conectado a MySQL y usando la base: {DB_NAME}")
        
        # Query para extraer solo las columnas que necesitas
        query = """
        SELECT 
            full_name,
            rut,
            base_wage,
            name_role,
            birthday,
            status,
            active_since
            
        FROM employees
        WHERE full_name IS NOT NULL 
        AND rut IS NOT NULL
        ORDER BY birthday
        """
        
        # Crear DataFrame directamente desde la query
        df = pd.read_sql(query, conexion)
        
        #print(f"✅ DataFrame creado exitosamente con {len(df)} registros")
        print(f"📋 Columnas disponibles: {list(df.columns)}")
        
        # Cerrar conexión
        conexion.close()
        print("✅ Conexión cerrada correctamente.")
        
        return df
        
    except Exception as e:
        print(f"❌ Error al conectar o extraer datos: {e}")
        return None

In [3]:
def calcular_edad_y_antiguedad(df):
    """
    Calcula la edad actual y los años de servicio en la empresa
    """
    if df is None or df.empty:
        return df
    
    print("Calculando edad y años de servicio...")
    fecha_actual = pd.Timestamp.now()
    
    # Calcular edad en años
    if 'birthday' in df.columns:
        df['edad_años'] = ((fecha_actual - df['birthday']).dt.days / 365.25).round(1)
        print("✅ Columna 'edad_años' creada")
       
    else:
        print("⚠️ Columna 'birthday' no encontrada - no se puede calcular edad")
    
    # Calcular años de servicio
    if 'active_since' in df.columns:
        df['años_servicio'] = ((fecha_actual - df['active_since']).dt.days / 365.25).round(1)
        print("✅ Columna 'años_servicio' creada")
        
        # Crear columna de antigüedad en categorías
        df['categoria_antiguedad'] = pd.cut(
            df['años_servicio'], 
            bins=[-float('inf'), 1, 3, 5, 10, float('inf')],
            labels=['Menos de 1 año', '1-3 años', '3-5 años', '5-10 años', 'Más de 10 años']
        )
        print("✅ Columna 'categoria_antiguedad' creada")
    else:
        print("⚠️ Columna 'active_since' no encontrada - no se puede calcular años de servicio")
    
    return df

In [4]:
# Ejecutar la extracción
if __name__ == "__main__":
    # Extraer los datos
    df_empleados = conectar_y_extraer_datos()

    # Para fechas
    df_empleados['birthday'] = pd.to_datetime(df_empleados['birthday'], errors='coerce')
    df_empleados['active_since'] = pd.to_datetime(df_empleados['active_since'], errors='coerce')

    # Para salarios (Int64 permite valores nulos)
    df_empleados['base_wage'] = df_empleados['base_wage'].astype('Int64')
    
    # ¡AQUÍ FALTABA ESTA LÍNEA IMPORTANTE!
    df_empleados = calcular_edad_y_antiguedad(df_empleados)
    
    # El DataFrame ya está listo para trabajar
    print("\n🎉 DataFrame 'df_empleados' listo con todas las columnas calculadas!")


🚀 Conectando a MySQL...
✅ Conectado a MySQL y usando la base: rrhh_app
📋 Columnas disponibles: ['full_name', 'rut', 'base_wage', 'name_role', 'birthday', 'status', 'active_since']
✅ Conexión cerrada correctamente.
Calculando edad y años de servicio...
✅ Columna 'edad_años' creada
✅ Columna 'años_servicio' creada
✅ Columna 'categoria_antiguedad' creada

🎉 DataFrame 'df_empleados' listo con todas las columnas calculadas!


C:\Users\gpavez\AppData\Local\Temp\ipykernel_8004\792895127.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conexion)


In [5]:
# Limpiar duplicados
print(f"📊 Registros antes de limpiar duplicados: {len(df_empleados)}")

# Ver duplicados por RUT (que debería ser único)
duplicados_rut = df_empleados[df_empleados.duplicated(subset=['rut'], keep=False)]
print(f"🔍 Registros con RUT duplicado: {len(duplicados_rut)}")

# Eliminar duplicados completos (todas las columnas iguales)
df_empleados = df_empleados.drop_duplicates()
print(f"✅ Registros después de eliminar duplicados completos: {len(df_empleados)}")

# Si aún hay duplicados por RUT, mantener solo el primero
df_empleados = df_empleados.drop_duplicates(subset=['rut'], keep='first')
print(f"✅ Registros después de eliminar duplicados por RUT: {len(df_empleados)}")

# Resetear el índice
df_empleados = df_empleados.reset_index(drop=True)
print("✅ Índice reseteado")

📊 Registros antes de limpiar duplicados: 3288
🔍 Registros con RUT duplicado: 3285
✅ Registros después de eliminar duplicados completos: 1098
✅ Registros después de eliminar duplicados por RUT: 1072
✅ Índice reseteado


In [6]:
df_empleados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1072 entries, 0 to 1071
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   full_name             1072 non-null   object        
 1   rut                   1072 non-null   object        
 2   base_wage             1071 non-null   Int64         
 3   name_role             1071 non-null   object        
 4   birthday              1072 non-null   datetime64[ns]
 5   status                1072 non-null   object        
 6   active_since          1072 non-null   datetime64[ns]
 7   edad_años             1072 non-null   float64       
 8   años_servicio         1072 non-null   float64       
 9   categoria_antiguedad  1072 non-null   category      
dtypes: Int64(1), category(1), datetime64[ns](2), float64(2), object(4)
memory usage: 77.8+ KB


In [7]:
def filtrar_empleados(df, años_min=None, cargo_texto=None, salario_min=None, salario_max=None):
    """
    Filtro múltiple para empleados
    """
    df_resultado = df.copy()
    filtros_aplicados = []
    
    # Filtro por antigüedad
    if años_min is not None:
        df_resultado = df_resultado[df_resultado['años_servicio'] >= años_min]
        filtros_aplicados.append(f"antigüedad >= {años_min} años")
    
    # Filtro por cargo
    if cargo_texto is not None:
        df_resultado = df_resultado[df_resultado['name_role'].str.contains(cargo_texto, case=False, na=False)]
        filtros_aplicados.append(f"cargo contiene '{cargo_texto}'")
    
    # Filtro por salario mínimo
    if salario_min is not None:
        df_resultado = df_resultado[df_resultado['base_wage'] >= salario_min]
        filtros_aplicados.append(f"salario >= ${salario_min:,}")
    
    # Filtro por salario máximo
    if salario_max is not None:
        df_resultado = df_resultado[df_resultado['base_wage'] <= salario_max]
        filtros_aplicados.append(f"salario <= ${salario_max:,}")
    
    print(f"🔍 Filtros aplicados: {', '.join(filtros_aplicados)}")
    print(f"📊 Resultados: {len(df_resultado)} empleados de {len(df)} total")
    
    return df_resultado